<a href="https://colab.research.google.com/github/records-sa/Python/blob/main/Colab_3%EC%A3%BC%EC%B0%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install dart-fss

In [ ]:
import dart_fss as dart_fss
import pandas as pd

api_key = '904e27f084e54f638fd2be6ba23b57fe74d70203'
dart_fss.set_api_key(api_key=api_key)


In [ ]:
all = dart_fss.api.filings.get_corp_code()

In [9]:
df = pd.DataFrame(all)

df_listed = df[df['stock_code'].notnull()]
df_non_listed = df[df['stock_code'].isnull()]

df_listed.to_excel('상장종목.xlsx')
df_non_listed.to_excel('비상장종목.xlsx')

In [55]:
def get_shareholders(corp_code):
  data = dart_fss.api.info.hyslr_sttus(corp_code, '2021', '11011')

  df = pd.DataFrame(data['list'])
  df = df[['corp_name','nm','relate','bsis_posesn_stock_qota_rt','trmend_posesn_stock_qota_rt','rm']]

  df.columns = ['기업명','이름','관계','기초지분율','기말지분율','비고']

  df = df[df['관계'].notnull()]

  df['기초지분율'] = pd.to_numeric(df['기초지분율'])
  df['기말지분율'] = pd.to_numeric(df['기말지분율'])

  return df.sort_values(by='기초지분율', ascending=False).head(3)

In [ ]:
corp_codes = list(df_listed.sample(10)['corp_code'])

dfs = []
for corp_code in corp_codes:
  try:
    df = get_shareholders(corp_code)
    dfs.append(df)
  except:
    print(f'error - {corp_code}')

df_result = pd.concat(dfs)
df_result['증감'] = df_result['기말지분율'] - df_result['기초지분율']

df_result.sort_values(by='증감', ascending=False)


In [ ]:
def get_salary(name):
  corp_code = df_listed[df_listed['corp_name'] == name].iloc[0,0]
  data = dart_fss.api.info.indvdl_by_pay(corp_code, '2021', '11011')

  df = pd.DataFrame(data['list'])

  df = df[['corp_name','nm','ofcps','mendng_totamt']]

  df.columns = ['기업명', '이름', '역할', '보수']

  df['보수'] = pd.to_numeric(df['보수'].str.replace(',',''))

  return df


In [ ]:
names = ['삼성전자','LG에너지솔루션','SK하이닉스','NAVER','삼성바이오로직스','삼성전자우','카카오','삼성SDI','현대차','LG화학','기아','POSCO홀딩스','KB금융','카카오뱅크','셀트리온','신한지주','삼성물산','현대모비스','SK이노베이션','LG전자','카카오페이','SK','한국전력','크래프톤','하나금융지주','LG생활건강','HMM','삼성생명','하이브','두산중공업','SK텔레콤','삼성전기','SK바이오사이언스','LG','S-Oil','고려아연','KT&G','우리금융지주','대한항공','삼성에스디에스','현대중공업','엔씨소프트','삼성화재','아모레퍼시픽','KT','포스코케미칼','넷마블','SK아이이테크놀로지','LG이노텍','기업은행']

dfs = []
for name in names:
  try:
    df = get_salary(name)
    dfs.append(df)
  except:
    print(f'error - {name}')

df_result = pd.concat(dfs)

df_result

In [ ]:
df_result.sort_values(by='보수', ascending=False).head(30)

In [69]:
def get_profit(name):
  corp_code = df_listed[df_listed['corp_name'] == name].iloc[0,0]
  data = dart_fss.api.finance.fnltt_singl_acnt(corp_code, '2021', '11011')
  df = pd.DataFrame(data['list'])

  cond = (df['fs_div'] == 'CFS') & (df['account_nm'] == '이익잉여금')
  df = df[cond]

  df['name'] = name

  df = df[['name','thstrm_amount','frmtrm_amount']]
  df.columns = ['기업명','당기','전기']

  df['당기'] = pd.to_numeric(df['당기'].str.replace(',',''))
  df['전기'] = pd.to_numeric(df['전기'].str.replace(',',''))

  df['증감'] = df['당기'] - df['전기']
  df['증감율'] = abs(df['증감']) / abs(df['전기'])

  return df

In [73]:
names = list(df_listed.sample(10)['corp_name'])

dfs = []
for name in names:
  try:
    df = get_profit(name)
    dfs.append(df)
  except:
    print(f'error - {name}')

df_result = pd.concat(dfs)
df_result.sort_values(by='증감율', ascending=False)

error - 키움제6호스팩
error - 그로웰전자
error - 부산도시가스
error - 유비온
error - 이노블루


,기업명,당기,전기,증감,증감율
7,쏠리드,38364485206,14509285028,23855200178,1.644133
7,동국제강,981476584895,448493672913,532982911982,1.188384
7,골드앤에스,-52116388794,-52886698849,770310055,0.014565
7,노루홀딩스,308296816940,311333116732,-3036299792,0.009753
